In [31]:
from __future__ import print_function
%matplotlib inline
import copy
import pandas as pd
import numpy as np
import librosa
import seaborn as sb
import matplotlib.pyplot as plt
import itertools
import re
import random
import gc
from os import listdir
from os.path import isfile, join
from numpy import median, diff
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Dropout, BatchNormalization
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier

In [2]:
samples_back_included = 8
num_classes = 5
num_features = 40
num_features_total = (num_features * samples_back_included) + 4
save_files = listdir('data')

def get_features_for_index(beat_features, notes, index):
    if index < 0:
        return [0] * num_features
    return beat_features[index]

def get_class_for_index(notes, index):
    if index < 0:
        return (0, 0)
    return notes[index][0].count('1')
    
importance_rankings = [48, 24, 12, 16, 6, 8, 3, 4, 2, 1]
def get_beat_importance(index):
    for i in range(len(importance_rankings)):
        if index % importance_rankings[i] == 0:
            return i

def get_features_for_song(key, is_full):
    X = []
    y = []
    if '{0}_beat_features.csv'.format(key) in save_files and '{0}_notes.csv'.format(key) in save_files:
        beat_features_rotated = pd.read_csv('data/{0}_beat_features.csv'.format(key)).values
        notes = pd.read_csv('data/{0}_notes.csv'.format(key), converters={'0': lambda x: str(x)}).values
        beat_features = np.flipud(np.rot90(np.array(beat_features_rotated)))
        num_notes = min(len(notes), len(beat_features))
        for i in range(num_notes):
            row_y = get_class_for_index(notes, i)
            if is_full or (not (row_y == 0 and random.randint(0, 20) != 0) and not (row_y == 1 and random.randint(0, 3) != 0)):
                features = [feature for j in range(samples_back_included) for feature in get_features_for_index(beat_features, notes, i - j)]
                features.append(i % 48)
                features.append(get_beat_importance(i))
                features.append(i / 48)
                features.append(num_notes - i / 48)
                X.append(features)
                y.append(row_y)
    return np.array(X), np.array(y)

def build_batch_generator():
    songs_to_use = pd.read_csv('data/songs_to_use.csv').values
    for song_data in songs_to_use:
        yield (get_features_for_song(song_data[0]))

# Total 243 songs
def build_training_data(songs_start, songs_end, is_full = False):
    X = []
    y = []
    songs_to_use = pd.read_csv('data/songs_to_use.csv').values
    for song_data in songs_to_use[songs_start:songs_end]:
        song_X, song_y = get_features_for_song(song_data[0], is_full)
        X.extend(song_X)
        y.extend(song_y)
    return X, y

In [3]:
# start with each input secion maps to one note
# train model for that (just list comprhension on noets for contains 1 maps to true)

# then move to bars eg section of 4 bars maps to output for each note
# error = probability of note being true vs was it really

# try bar + prev notes (home use weird dimensioned data?) to predict next notes

# try feeding in non structured data (bpm, position of time in song, song length, 
# things about feel of song (generated features))
# try using keras merge layer to add extra features

In [4]:
# make two separate classifiers, one for note importance, then another to take importance and previous notes/importance and 
# output the notes

# can either go back and include prev samples to get importance or atual notes

In [5]:
# Try building one network to output importance of beats, another to output song from that and prev notes
# Also start building sequentially not off of stepfile and test on new songs

In [7]:
X_train, y_train = build_training_data(0, 243)
#X_test, y_test = build_training_data(200, 243)

In [15]:
X_train_full, y_train_full = build_training_data(0, 243, True)
X_test_full, y_test_full = build_training_data(200, 243, True)

In [28]:
beat_feature_model = Sequential()

beat_feature_model.add(Dense(500, input_dim=num_features_total, init='uniform'))
beat_feature_model.add(BatchNormalization())
beat_feature_model.add(Activation('tanh'))
beat_feature_model.add(Dropout(0.5))

beat_feature_model.add(Dense(500, init='uniform'))
beat_feature_model.add(BatchNormalization())
beat_feature_model.add(Activation('tanh'))
beat_feature_model.add(Dropout(0.5))

beat_feature_model.add(Dense(500, init='uniform'))
beat_feature_model.add(BatchNormalization())
beat_feature_model.add(Activation('tanh'))
beat_feature_model.add(Dropout(0.5))

#beat_feature_model.add(Dense(1, init='uniform'))
beat_feature_model.add(Dense(num_classes, init='uniform'))
beat_feature_model.add(BatchNormalization())
beat_feature_model.add(Activation('softmax'))

#beat_feature_model.compile(loss='mean_squared_error',
beat_feature_model.compile(loss='categorical_crossentropy',
                           optimizer='adadelta',
                           metrics=['accuracy'])

In [9]:
y_one_hot = np.zeros((len(y_train), num_classes))
y_one_hot[np.arange(len(y_train)), y_train] = 1

beat_feature_model.fit(np.array(X_train), np.array(y_one_hot), nb_epoch=7, batch_size=10) #, class_weight=class_weight)
beat_feature_model.save('models/beat_importance_model.h5')

Epoch 1/10
49430/49430 [==============================] - 53s - loss: 1.1318 - acc: 0.5764    
Epoch 2/10
49430/49430 [==============================] - 53s - loss: 0.8116 - acc: 0.6701    
Epoch 3/10
49430/49430 [==============================] - 53s - loss: 0.7661 - acc: 0.6784    
Epoch 4/10
49430/49430 [==============================] - 54s - loss: 0.7539 - acc: 0.6830    
Epoch 5/10
49430/49430 [==============================] - 55s - loss: 0.7483 - acc: 0.6836    
Epoch 6/10
49430/49430 [==============================] - 55s - loss: 0.7442 - acc: 0.6819    
Epoch 7/10
49430/49430 [==============================] - 57s - loss: 0.7409 - acc: 0.6829    
Epoch 8/10
49430/49430 [==============================] - 64s - loss: 0.7362 - acc: 0.6856    
Epoch 9/10
49430/49430 [==============================] - 47s - loss: 0.7355 - acc: 0.6855    
Epoch 10/10
49430/49430 [==============================] - 46s - loss: 0.7334 - acc: 0.6853    


In [29]:
y_one_hot = np.zeros((len(y_train_full), num_classes))
y_one_hot[np.arange(len(y_train_full)), y_train_full] = 1

beat_feature_model.fit(np.array(X_train_full), np.array(y_one_hot), nb_epoch=3, batch_size=100) #, class_weight=class_weight)
beat_feature_model.save('models/beat_importance_full_model.h5')

Epoch 1/3
601265/601265 [==============================] - 218s - loss: 1.3761 - acc: 0.6271   
Epoch 2/3
601265/601265 [==============================] - 138s - loss: 0.9343 - acc: 0.8105   
Epoch 3/3
601265/601265 [==============================] - 138s - loss: 0.6548 - acc: 0.8506   


In [32]:
beat_feature_model = load_model('models/beat_importance_model.h5')

In [33]:
def calculate_importance(row):
    return (1 - row[0]) * (row[1] + row[2] * 2 + row[3] * 30 + row[4] * 40)

In [34]:
def step_song(key, clf):
    song_X, song_y = get_features_for_song(key, True)
    new_song_y = clf.predict(song_X)
    beat_importance = [calculate_importance(row) for row in new_song_y]
    
    #print ('Length: ' + str(len(new_song_y)))
    #plt.plot([new_song_y[i] for i in range(len(new_song_y)) if i % 12 == 0])
    #plt.show()
    
    pd.DataFrame(beat_importance).to_csv('generated_data/{0}_importance_generated.csv'.format(key), index=False)

In [ ]:
songs_to_use = pd.read_csv('data/songs_to_use.csv').values
for song_data in songs_to_use:
    try:
        step_song(song_data[0], beat_feature_model)
    except:
        print ('\nError loading song')
        print (song_data[0])


Error loading song
In The Groove~I Think I Like That Sound

Error loading song
In The Groove~Remember December

Error loading song
In The Groove~Torn

Error loading song
In The Groove~Walking on Fire

Error loading song
In The Groove 2~!
